___
# Ciência dos Dados - PROJETO 2

___
## Nome 1
#### Gabriel Couto

## Nome 2
#### Gabriel Miras

## Nome 3
#### Mariana Abrantes

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [51]:
import pandas as pd
import os
import random
import re

In [11]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Mariana Abrantes\Documents\ciencia_dados\DP-P2


In [35]:
leitura = pd.ExcelFile('spamham2019(1).xlsx')
dados = pd.read_excel(leitura)
dados.head()

,Email,Class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [59]:
dados_treinamento = dados.sample(int(0.75 * len(dados)))
#treinamento.head()

In [60]:
dados_teste = dados.sample(int(0.25 * len(dados)))
#teste.head()

# Limpando a base de dados

In [64]:
tirar = [':',',',';','!','@','#','$','%','¨','&','*','(',')','-','+','{','}', 'å']
for i in tirar:
    dados_treinamento.Email = dados_treinamento.Email.str.replace(i, '', regex=False)

AttributeError: 'Series' object has no attribute 'Email'

___
## 3. Classificador Naive-Bayes

___
## 4. Qualidade do Classificador alterando a base de treinamento